In [1]:
import numpy as np
import os
import json
import re
from gensim.models import KeyedVectors

import string
from collections import Counter
import operator

In [11]:
a=[10,10,11]
import sys
sys.getsizeof(a)

88

In [2]:
model= KeyedVectors.load_word2vec_format('../Database/frWac_non_lem_no_postag_no_phrase_200_skip_cut100.bin', binary=True)

In [3]:
table = str.maketrans(string.punctuation, ' '*len(string.punctuation))
def remove_tag(text):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', text)
    return cleantext

def get_counters(text):
    text_without_tag=remove_tag(text)
    text_without_tag=''.join([i for i in text_without_tag if not i.isdigit()])
    text_split=text_without_tag.translate(table).lower().replace('\n', ' ').split(' ')    
    counter_text=Counter(text_split)
    return counter_text

In [5]:
path='/home/gabriel/Documents/MPRI/Web_Data_Management/wikiextractor-master/text/'
 #map punctuation to space
voc_dict={}
length_voc=0

for w,i in enumerate(os.listdir(path)):
    for j in os.listdir(path+i):
        for filename in os.listdir(path+i+'/'+j):
            with open(path+i+'/'+j+'/'+filename) as f:
                lines = [line.rstrip('\n') for line in f]
            for line_index,line in enumerate(lines):
                a=json.loads(line)
                if 'text' in a:
                    counter_text=get_counters(a['text'])
                    for key in counter_text.keys():
                        if key not in voc_dict:
                            voc_dict[key]=counter_text[key]
                        else:
                            voc_dict[key]+=counter_text[key]
                        length_voc+=counter_text[key]
                if length_voc>1000000:
                    s=q

NameError: name 'q' is not defined

In [6]:
for key in voc_dict.keys():
    voc_dict[key]/=length_voc

In [16]:
j = json.dumps(voc_dict)
f = open("../Database/voc_dict.json","w")
f.write(j)
f.close()

In [7]:
def modify_word(word):
    letters=list("abcdefghijklmnopqrstuvwxyzéèàêûöù")
    if type(word)==str:
        word_list=list(word)
        
    modify_1_letter=[]
    for i in range(0,len(word_list)):
        for j in range(len(letters)):
            tmp=list(word)
            tmp[i]=letters[j]
            modify_1_letter.append(tmp)

    swap_one_letter=[]
    for i in range(0,len(word_list)-1):
        tmp=list(word)
        a=word_list[i]
        tmp[i]=tmp[i+1]
        tmp[i+1]=a
        swap_one_letter.append(tmp)
    
    insert_one_letter=[]
    for i in range(0,len(word_list)+1):
        for j in range(len(letters)):
            tmp=list(word)
            tmp.insert(i,letters[j])
            insert_one_letter.append(tmp)
    
    missing_one_letter=[]
    for i in range(0,len(word_list)):
        tmp=list(word)
        del tmp[i]
        missing_one_letter.append(tmp)
    
    return [''.join(i) for i in modify_1_letter+swap_one_letter+insert_one_letter+missing_one_letter]

In [100]:
def distance(a,b):
    return np.mean((a.reshape(1,-1)-b.reshape(1,-1))**2)

somme=lambda x:sum([i[1] for i in x])

query_sentence="Le Soleil est une étole naine jaue qui se compose"
query_sentence_ori=query_sentence.lower().replace('\n', ' ').split(' ')
query_sentence=query_sentence.translate(table).lower().replace('\n', ' ').split(' ')
already_in=[i for i in query_sentence if i in voc_dict]
w2v_query=np.mean(np.array([model[i] for i in already_in if i in model]),axis=0)
query_sentence=[i for i in query_sentence if i not in voc_dict]
for query in query_sentence:
    l=[modify_word(query)]+[modify_word(i) for i in set(modify_word(query))]
    reco = set([item for sublist in l for item in sublist if item in voc_dict])
    dict1 = Counter(query) 
    reco_list=[]
    for i in reco:
        dict2 = Counter(i)
        commonDict = dict1 & dict2
        diff1=len(query)/np.sum([value for key,value in commonDict.items()])
        diff2=abs(len(query)-len(i))
        reco_list.append([i,voc_dict[i]*(1/diff1)*(1/(1+diff2))])
    reco_list=sorted(reco_list,key=lambda x:x[1],reverse=True)
    reco_list=[[i[0],float(i[1])/somme(reco_list)] for i in reco_list]
    w2v=[[i[0],distance(model[i[0]],w2v_query)] for i in reco_list if i[0] in model]
#     w2v=sorted(w2v,key=lambda x:x[1])
    w2v=[[i[0],i[1]/somme(w2v)] for i in w2v]
#     print(w2v)
#     print()
#     print(reco_list)
    both=sorted([[w2v[i][0],-0.8*w2v[i][1]+0.2*reco_list[i][1]] for i in range(len(w2v))],key=lambda x:x[1],reverse=True)
#     print()
#     print(both)
#     print()
    query_sentence_ori=[i if i!=query else both[0][0] for i in query_sentence_ori]
print(' '.join(query_sentence_ori))

le soleil est une école saint au qui se compose


In [96]:
query_sentence_ori

['theorie', 'des', 'grahes']